In [1]:
import geopandas as gpd
import rasterio
import numpy as np
from shapely.geometry import Point
from rasterio.mask import mask

#load gpkg generated from 5-Postprocessing-FillHole-DelArea.ipynb
# vector_gdf = gpd.read_file(r'I:\ACM\FinalOutPut\PredictLake_0603_postprocessed.gpkg') 
# vector_gdf = gpd.read_file(r'I:\ACM\FinalOutPut\PredictLake_0619_postprocessed.gpkg') 
# vector_gdf = gpd.read_file(r'I:\ACM\FinalOutPut\PredictLake_0731_postprocessed.gpkg') 
vector_gdf = gpd.read_file(r'I:\ACM\FinalOutPut\PredictLake_0825_postprocessed.gpkg') 

#load tif
# tif_file = r'I:\ACM\Greenland26X_22W_Sentinel2_2019-06-03_05.tif'
# tif_file = r'I:\ACM\Greenland26X_22W_Sentinel2_2019-06-19_20.tif'
# tif_file = r'I:\ACM\Greenland26X_22W_Sentinel2_2019-07-31_25.tif'
tif_file = r'I:\ACM\Greenland26X_22W_Sentinel2_2019-08-25_29.tif'



# Define a function to retrieve RGB values at specified coordinates
def get_rgb_values(x, y):
    # Use Rasterio to read band values at the specified coordinates
    tif_data = src.sample([(x, y)])
    tif_values = next(tif_data)
    r = tif_values[0]
    g = tif_values[1]
    b = tif_values[2]
    return r, g, b


# Open the tif image
with rasterio.open(tif_file) as src:
    # Loop through each feature
    for index, row in vector_gdf.iterrows():
        try:
            # Get the geometry of the feature
            geom = row['geometry']

            # get the bounding rectangle of each feature
            poly_box = geom.minimum_rotated_rectangle
            x_box, y_box = poly_box.exterior.coords.xy
            edge_length = (Point(x_box[0], y_box[0]).distance(Point(x_box[1], y_box[1])), Point(x_box[1], y_box[1]).distance(Point(x_box[2], y_box[2])))
            length = max(edge_length)/1e3
            width = min(edge_length)/1e3
            aspectRatio = length/width

            # Buffer the geometry by 100 meters
            buffered_geom = geom.buffer(100)

            # Simplify the boundary of the buffered geometry with a tolerance of 10 (you can adjust as needed)
            simplified_geom = buffered_geom.simplify(10)

            # Calculate the average RGB values for all points within the simplified geometry
            r_sum = 0.0
            g_sum = 0.0
            b_sum = 0.0
            point_count = 0
            
            for x, y in simplified_geom.exterior.coords:
                r, g, b = get_rgb_values(x, y)
                r_sum += r
                g_sum += g
                b_sum += b
                point_count += 1

            if point_count > 0:
                r_avg = r_sum / point_count
                g_avg = g_sum / point_count
                b_avg = b_sum / point_count
            else:
                r_avg, g_avg, b_avg = 0, 0, 0

            # Use rasterio's mask function to extract image data within the feature's boundary
            out_image, out_transform = mask(src, [geom], nodata=0, crop=True)

            # Calculate the mean values for each band
            out_image = out_image.astype('float')
            out_image = np.where(out_image!=0, out_image, np.nan)
            r_band = out_image[0]  # R band
            g_band = out_image[1]  # G band
            b_band = out_image[2]  # B band

            # Calculate the percentage of red pixel whose value<=150 in a lake
            r_polygon_sum = np.sum(r_band, where=~np.isnan(r_band))
            r_band_le150 = np.where(r_band<=150, r_band, np.nan)
            r_le150_sum = np.sum(r_band_le150, where=~np.isnan(r_band_le150))
            r_le150_ratio = r_le150_sum / r_polygon_sum

            r_polygon_mean = np.nanmean(r_band)
            b_polygon_mean = np.nanmean(b_band)
            r_polygon_min = np.nanmin(r_band)
            g_polygon_min = np.nanmin(g_band)
            b_polygon_min = np.nanmin(b_band)
            
            # Update the attributes of the vector feature
            vector_gdf.at[index, 'R_boundary'] = r_avg
            vector_gdf.at[index, 'G_boundary'] = g_avg
            vector_gdf.at[index, 'B_boundary'] = b_avg
            vector_gdf.at[index, 'R_B_diff_polygon_mean'] = r_polygon_mean - b_polygon_mean
            vector_gdf.at[index, 'R_polygon_min'] = r_polygon_min
            vector_gdf.at[index, 'G_polygon_min'] = g_polygon_min
            vector_gdf.at[index, 'B_polygon_min'] = b_polygon_min     
            vector_gdf.at[index, 'R_polygon_le150_ratio'] = r_le150_ratio
            vector_gdf.at[index, 'polygon_aspectRatio'] = aspectRatio
        except:
            pass

C:\Users\authu\AppData\Local\Temp\ipykernel_61632\4121061921.py:88: RuntimeWarning: invalid value encountered in double_scalars
  r_le150_ratio = r_le150_sum / r_polygon_sum
C:\Users\authu\AppData\Local\Temp\ipykernel_61632\4121061921.py:90: RuntimeWarning: Mean of empty slice
  r_polygon_mean = np.nanmean(r_band)
C:\Users\authu\AppData\Local\Temp\ipykernel_61632\4121061921.py:91: RuntimeWarning: Mean of empty slice
  b_polygon_mean = np.nanmean(b_band)
C:\Users\authu\AppData\Local\Temp\ipykernel_61632\4121061921.py:92: RuntimeWarning: All-NaN slice encountered
  r_polygon_min = np.nanmin(r_band)
C:\Users\authu\AppData\Local\Temp\ipykernel_61632\4121061921.py:93: RuntimeWarning: All-NaN slice encountered
  g_polygon_min = np.nanmin(g_band)
C:\Users\authu\AppData\Local\Temp\ipykernel_61632\4121061921.py:94: RuntimeWarning: All-NaN slice encountered
  b_polygon_min = np.nanmin(b_band)
C:\Users\authu\AppData\Local\Temp\ipykernel_61632\4121061921.py:88: RuntimeWarning: invalid value encoun

In [3]:
len(vector_gdf.loc[vector_gdf['polygon_aspectRatio']>10])

20

In [5]:
# Delete features that located at slush fields, water pixel (red band pixel whose value<=150) number that less than 1% of valid pixel number in the lake are excluded
vector_gdf_cp = vector_gdf.copy()
index_noSlushField = vector_gdf_cp.loc[~(vector_gdf_cp['R_polygon_le150_ratio']>=0.01)].index
vector_gdf_cp.drop(index=index_noSlushField, inplace=True)

# Delete non-supraglacial features
index_noSupraglacial = vector_gdf_cp.loc[(vector_gdf_cp['R_boundary']<150) & (vector_gdf_cp['G_boundary']<150) & (vector_gdf_cp['B_boundary']<150)].index
vector_gdf_cp.drop(index=index_noSupraglacial, inplace=True)
index_noSupraglacialEdge = vector_gdf_cp.loc[vector_gdf_cp['R_B_diff_polygon_mean']>=-10].index #pixels of R-B>=-10 locate at the supraglacial edge
vector_gdf_cp.drop(index=index_noSupraglacialEdge, inplace=True)

# Filter narrow streams with aspect ratio of 10:1       
# ATTENTION!!! Use this filter only if those "narrow streams" were excluded in your test labels, otherwise it can influence our F1 score!!!

index_noNarrowStream = vector_gdf_cp.loc[vector_gdf_cp['polygon_aspectRatio']>10].index
vector_gdf_cp.drop(index=index_noNarrowStream, inplace=True)

vector_gdf_cp.reset_index(drop=True, inplace=True)

# outputpackage=r'I:\ACM\FinalOutPut\PredictLake_0825_final_withStreams.gpkg'
outputpackage=r'I:\ACM\FinalOutPut\PredictLake_0825_final_withoutStreams.gpkg'
vector_gdf_cp.to_file(outputpackage, driver='GPKG')
vector_gdf_cp

,level_0,level_1,geometry,R_boundary,G_boundary,B_boundary,R_B_diff_polygon_mean,R_polygon_min,G_polygon_min,B_polygon_min,R_polygon_le150_ratio,polygon_aspectRatio
0,0,638,"POLYGON ((-5591525.610 9810122.054, -5591525.3...",144.160000,148.600000,151.600000,-87.449153,12.0,81.0,107.0,1.000000,1.163115
1,0,744,"POLYGON ((-5512054.502 9640934.395, -5512050.9...",200.380952,206.285714,208.476190,-37.867596,89.0,125.0,123.0,0.904040,1.289660
2,0,745,"POLYGON ((-5517561.710 9641888.269, -5517557.9...",218.868421,225.526316,228.302632,-46.396900,1.0,17.0,50.0,0.973089,1.976785
3,0,756,"POLYGON ((-5506331.443 9656600.531, -5506300.4...",217.068966,221.827586,224.068966,-45.325000,91.0,127.0,129.0,0.955467,1.944966
4,0,759,"POLYGON ((-5531905.468 9685642.568, -5531905.3...",196.962963,208.518519,212.555556,-50.058824,97.0,148.0,157.0,0.883236,2.089429
...,...,...,...,...,...,...,...,...,...,...,...,...
478,0,2306,"POLYGON ((-2569406.765 14729310.891, -2569403....",182.238095,191.904762,196.523810,-43.846809,47.0,63.0,71.0,0.956292,3.794304
479,0,2308,"POLYGON ((-2557040.718 14740314.470, -2557037....",183.045455,206.363636,212.727273,-57.280000,125.0,178.0,193.0,0.679802,1.389803
480,0,2309,"POLYGON ((-2558187.505 14745282.899, -2558183....",205.588235,217.000000,220.441176,-50.015873,88.0,160.0,178.0,0.452208,1.551686
481,0,2310,"POLYGON ((-2578339.121 14745891.032, -2578338....",216.666667,224.939394,227.878788,-52.972686,121.0,183.0,200.0,0.281102,1.730912
